# file_system

In [ ]:
open rust_operators
open sm'_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    ()

()



## (</>)

In [ ]:
inl (</>) (a : string) (b : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.Combine (!a, !b)'
        | target => fun () => failwith $'$"(</>) / target: {!target}"'

()



## create_temp_directory_name

In [ ]:
inl create_temp_directory_name () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl root = $'System.Reflection.Assembly.GetEntryAssembly().GetName().Name' : string

            ($'System.IO.Path.GetTempPath' () : string)
            </> ($'$"\!{!root}"' : string)
            </> (date_time.new_guid_from_date_time $'System.DateTime.Now' |> sm'.obj_to_string)
        | target => fun () => failwith $'$"create_temp_directory_name / target: {!target}"'

()



In [ ]:
// // test

create_temp_directory_name ()
|> _contains ($'System.IO.Path.DirectorySeparatorChar' : char)

type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1 of f1_0 : US0
and [<Struct>] US2 =
    | US2_0 of f0_0 : char
    | US2_1
let rec closure0 (v0 : char) (v1 : char) : bool =
    let v2 : bool = v0 = v1
    v2
and closure1 () (v0 : char) : US2 =
    US2_0(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string option = None
    let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM
    let v1 : US0 = US0_0
    let v2 : US1 = US1_0(v1)
    let v3 : string = $"create_temp_directory_name / target: {v2}"
    let v4 : string = failwith<string> v3
    v4
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v5 : US0 = US0_1
    let v6 : US1 = US1_0(v5)
    let v7 : string = $"create_temp_directory_name / target: {v6}"
    let v8 : string = failwith<string> v7
    v8
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !WASM
    let v9 : string = System.Reflection.Assembly.GetEntryAs

## directory_info

In [ ]:
nominal directory_info =
    // $'\n#if \!FABLE_COMPILER && \!FABLE_COMPILER_RUST && \!WASM\n                                                                                                                   System.IO.DirectoryInfo\n#else\n                                                                                                                   _\n#endif\n                                                                                                                  '
    $'System.IO.DirectoryInfo'

()



## directory_info_exists

In [ ]:
inl directory_info_exists (info : directory_info) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Exists'
        | target => fun () => failwith $'$"directory_info_exists / target: {!target}"'

()



## directory_info_creation_time

In [ ]:
inl directory_info_creation_time (info : directory_info) : date_time.date_time =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.CreationTime'
        | target => fun () => failwith $'$"directory_info_creation_time / target: {!target}"'

()



## create_directory

In [ ]:
inl create_directory (path : string) : directory_info =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.CreateDirectory'
        | target => fun () => failwith $'$"create_directory / target: {!target}"'

()



## directory_exists

In [ ]:
inl directory_exists (path : string) : bool =
    path |> $'System.IO.Directory.Exists'

()



## create_temp_directory

In [ ]:
inl create_temp_directory () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl temp_folder = create_temp_directory_name ()
            inl result = temp_folder |> create_directory

            inl exists' = result |> directory_info_exists
            if not exists' then
                inl creation_time = result |> directory_info_creation_time
                inl result = ($'{| Exists = !exists'; CreationTime = !creation_time |}' : any) |> sm'.format_debug
                inl get_locals () : string =
                    $'$"temp_folder: {!temp_folder} / result: {!result} {!get_locals ()}"'
                // trace Debug (fun () => "createTempDirectory") get_locals
                console.write_line ($'$"create_temp_directory / {!get_locals ()}"' : string)

            temp_folder
        | target => fun () => failwith $'$"create_temp_directory / target: {!target}"'

()



In [ ]:
// // test

create_temp_directory ()
|> directory_exists
|> _assert_eq true

type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1 of f1_0 : US0
let rec closure1 () () : string =
    let v0 : string = ""
    v0
and closure0 (v0 : string, v1 : string) () : string =
    let v2 : (unit -> string) = closure1()
    let v3 : string = $"temp_folder: {v0} / result: {v1} {v2 ()}"
    v3
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string option = None
    let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM
    let v1 : US0 = US0_0
    let v2 : US1 = US1_0(v1)
    let v3 : string = $"create_temp_directory / target: {v2}"
    let v4 : string = failwith<string> v3
    v4
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v5 : US0 = US0_1
    let v6 : US1 = US1_0(v5)
    let v7 : string = $"create_temp_directory / target: {v6}"
    let v8 : string = failwith<string> v7
    v8
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !WASM
    let v9 : string option =

## main

In [ ]:
inl main () =
    $"let create_temp_directory_name () = !create_temp_directory_name ()" : ()
    $"let create_temp_directory () = !create_temp_directory ()" : ()
    inl combine x = (</>) x
    $"let (</>) x = !combine x" : ()

type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1 of f1_0 : US0
let rec closure0 () () : string =
    let v0 : string option = None
    let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM
    let v1 : US0 = US0_0
    let v2 : US1 = US1_0(v1)
    let v3 : string = $"create_temp_directory_name / target: {v2}"
    let v4 : string = failwith<string> v3
    v4
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v5 : US0 = US0_1
    let v6 : US1 = US1_0(v5)
    let v7 : string = $"create_temp_directory_name / target: {v6}"
    let v8 : string = failwith<string> v7
    v8
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !WASM
    let v9 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    let v10 : (unit -> string) = System.IO.Path.GetTempPath
    let v11 : string = v10 ()
    let v12 : string = $"!{v9}"
    let v13 : string option = None
    let mutable _v13 = v13
    #if FABLE_COM